<a href="https://colab.research.google.com/github/hongo-daisuke/study-python/blob/master/python_database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# データーベース

## リレーショナルデータベース
### SQL文で様々なことが可能

*   SQLite
*   MySQL



### SQLite
`brew install sqlite3`


```
# コンソール上でのsqliteへのアクセス
sqlite3 DB名.db

# テーブルの一覧確認
.tables
```



In [25]:
import sqlite3

conn = sqlite3.connect('test.db')

# 何度もSQLを実行したい場合は下記の通りに記述する
# conn = sqlite3.connect(':memory:')

# sqliteを操作するカーソルを準備
curs = conn.cursor()

In [17]:
# テーブルの作成
curs.execute(
    'CREATE TABLE persons(id INTEGER PRIMARY KEY AUTOINCREMENT, name STRING)'
    )

# コミットでDBに反映
conn.commit()

In [18]:
# データをInsertする
curs.execute(
    "INSERT INTO persons(name) values('John')"
    )

# コミットでDBに反映
conn.commit()

In [19]:
# データをInsertする
curs.execute(
    "INSERT INTO persons(name) values('Bob')"
    )
curs.execute(
    "INSERT INTO persons(name) values('James')"
    )

# コミットでDBに反映
conn.commit()

In [20]:
# データの検索
curs.execute(
    'SELECT * FROM persons'
    )
r = curs.fetchall()

print(r)

[(1, 'John'), (2, 'Bob'), (3, 'James')]


In [22]:
# データの更新
# JohnをMichaelに更新
curs.execute(
    "UPDATE persons SET name = 'Michael' WHERE name = 'John'"
    )

# コミットでDBに反映
conn.commit()

# データの検索
curs.execute(
    'SELECT * FROM persons'
    )
r = curs.fetchall()

print(r)

[(1, 'Michael'), (2, 'Bob'), (3, 'James')]


In [23]:
# データの削除
curs.execute(
    "DELETE FROM persons WHERE name = 'Bob'"
    )

# コミットでDBに反映
conn.commit()

# データの検索
curs.execute(
    'SELECT * FROM persons'
    )
r = curs.fetchall()

print(r)

[(1, 'Michael'), (3, 'James')]


In [24]:
# カーソルもクローズする
curs.close()

# sqliteへのコネクションをクローズする
conn.close()

### MySQL

```
# MacへのMysqlインストール
brew install mysql

# ドライバインストール
pip3 install mysql-connector-python

# 起動
brew services start mysql

# 停止
brew services stop mysql
```



In [ ]:
import mysql.connector

# Mysqlに接続
#conn = mysql.connector.connect(host='127.0.0.1', user='root')

# 接続するDBを指定する
conn = mysql.connector.connect(host='127.0.0.1', user='root', database='test_db')

# mysqlを操作するカーソルを準備
curs = conn.cursor()

# DBの作成
curs.execute(
    'CREATE DATABASE test_db'
    )

# テーブルの作成
curs.execute(
    'CREATE TABLE persons('
    'id int NOT NULL AUTO_INCREMENT,'
    'name varchar(14) NOT NULL,'
    'PRIMARY KEY(id))'
    )

# コミットでDBに反映
conn.commit()

# データをInsertする
curs.execute(
    "INSERT INTO persons(name) values('John')"
    )

# データの検索
curs.execute(
    'SELECT * FROM persons'
    )
for row in curs:
    print(row)


# データの更新
curs.execute(
    "UPDATE persons SET name = 'John' WHERE name = 'Mike'"
    )
# コミットでDBに反映
conn.commit()


# データの削除
curs.execute(
    "DELETE FROM persons WHERE name = 'Mike'"
    )
# コミットでDBに反映
conn.commit()


# カーソルをクローズする
curs.close()

# mysqlへのコネクションをクローズする
conn.close()

### SQLAlchemy

```
pip3 install sqlalchemy
pip3 install pymysql
```



#### sqlalchemy + sqlite

In [1]:
import sqlalchemy
import sqlalchemy.ext.declarative
import sqlalchemy.orm

# echo=Trueを引数に含めるとどのようなSQLを実行したか出力される
# engine = sqlalchemy.create_engine('sqlite:///:memory:', echo=True) # メモリー上で実行
engine = sqlalchemy.create_engine('sqlite:///test_sqlite.db', echo=True) # SQLiteに書き込む

Base = sqlalchemy.ext.declarative.declarative_base()

class Person(Base):
    __tablename__ = 'persons'
    id = sqlalchemy.Column(
        sqlalchemy.Integer, primary_key=True, autoincrement=True
    )
    name = sqlalchemy.Column(
        sqlalchemy.String(14)
    )

# sqlのエンジンにBaseで作ったメタデータ(Personクラス)の型をしたテーブルSQLiteに書き込む
Base.metadata.create_all(engine)

# データベースにアクセス
Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

# データの書き込み
p1 = Person(name='John')
session.add(p1)
p2 = Person(name='Bob')
session.add(p2)
p3 = Person(name='Mary')
session.add(p3)
session.commit()

# 指定してデータの取得
p4 = session.query(Person).filter_by(name='Bob').first()
# データの更新
p4.name = 'James'
session.add(p4)
session.commit()


# 指定してデータの取得
p5 = session.query(Person).filter_by(name='John').first()
# データの削除
p5.name = 'James'
session.delete(p5)
session.commit()


# データの読み込み
persons = session.query(Person).all()
for person in persons:
    print(person.id, person.name)

2 James
3 Mary


#### sqlalchemy + mysql

In [ ]:
import sqlalchemy
import sqlalchemy.ext.declarative
import sqlalchemy.orm

# mysqlclientの場合
#engine = sqlalchemy.create_engine('mysql://<user>:<password>@<host>[:<port>]/<dbname>')

# mysql-connector-pythonの場合
#engine = sqlalchemy.create_engine('mysql+mysqlconnector://root:@localhost/test_mysql_sqlalchemy', echo=False)

# pymysqlの場合
engine = sqlalchemy.create_engine('mysql+pymysql://root:@localhost/test_mysql_sqlalchemy', echo=False)

Base = sqlalchemy.ext.declarative.declarative_base()

class Person(Base):
    __tablename__ = 'persons'
    id = sqlalchemy.Column(
        sqlalchemy.Integer, primary_key=True, autoincrement=True
    )
    name = sqlalchemy.Column(
        sqlalchemy.String(14)
    )

# sqlのエンジンにBaseで作ったメタデータ(Personクラス)の型をしたテーブルをMysqlに書き込む
Base.metadata.create_all(engine)

# データベースにアクセス
Session = sqlalchemy.orm.sessionmaker(bind=engine)
session = Session()

# データの書き込み
p1 = Person(name='John')
session.add(p1)
p2 = Person(name='Bob')
session.add(p2)
p3 = Person(name='Mary')
session.add(p3)
session.commit()

# 指定してデータの取得
p4 = session.query(Person).filter_by(name='Bob').first()
# データの更新
p4.name = 'James'
session.add(p4)
session.commit()


# 指定してデータの取得
p5 = session.query(Person).filter_by(name='John').first()
# データの削除
p5.name = 'James'
session.delete(p5)
session.commit()


# データの読み込み
persons = session.query(Person).all()
for person in persons:
    print(person.id, person.name)

## NoSQL

*   DBM、memcach
*   MongoDB
*   Hbase
*   Neo4j



### DBM、memcach
キーバーリュー型
*   webサイトのデータベースにアクセスする前にメモリ上にデータベースを作成し簡易的に取得し、Webサイトの高速化を行う



### DBM

In [1]:
import dbm

# DBファイル作成と書き込み
with dbm.open('cache', 'c') as db:
    # int型は入れることが出来ないので注意
    db['key1'] = 'value1'
    db['key2'] = 'value2'

In [2]:
# 値の取り出し
with dbm.open('cache', 'r') as db:
    print(db.get('key2'))

b'value2'


#### memcach
データとオブジェクトをメモリー内にキャッシュさせてデータベースへのアクセス回数を減少させ、Webサイトの高速化を行う


```
brew install memcached

pip3 install python-memcached
```



In [ ]:
import memcache

# memcacheのクライアントを作成
db = memcache.Client(['127.0.0.1:11211'])

# 値のセット
db.set('web_page', 'value1')

# 値の保持時間を秒単位で設定可能
db.set('web_page', 'value1', time=2)

# 値の取得
r = db.get('web_page')
print(r)

# カウンター
db.set('count', 0)
db.incr('count', 1)
db.incr('count', 1)
db.incr('count', 1)
r = db.get('count')
print(r)

#### memcache + sqlite

In [ ]:
import sqlite3
import memcache
import pathlib
import os


def setting(conn):

    # テーブルの作成
    curs.execute(
        'CREATE TABLE persons(employ_id INTEGER PRIMARY KEY AUTOINCREMENT, name STRING)'
        )

    # データをInsertする
    curs.execute(
        "INSERT INTO persons(name) values('James')"
        )

    # コミットでDBに反映
    conn.commit()
    
    pathlib.Path('check.txt').touch()


def get_employ_id(name):
    # memcacheよりemploy_idを取得
    employ_id = db.get(name)
    
    # employ_idが取得出来た場合はそのまま返却
    if employ_id:
        return employ_id
      
    # memcacheに存在しない場合、sqliteから取得
    curs.execute(
        "SELECT * FROM persons WHERE name = '{}'".format(name)
        )
    
    # fetchone()で1件取得
    person = curs.fetchone()
    
    # 取得出来なかった場合はエラー
    if not person:
      raise Exception('No employ')
    
    # タプルで取得されるので格変数に代入
    employ_id, name = person
    
    # memcacheに値をセットする
    db.set(name, employ_id, time=120)
    return employ_id


# memcacheのクライアントを作成
db = memcache.Client(['127.0.0.1:11211'])

# sqliteにアクセス
conn = sqlite3.connect('memcache_sqlite3.db')

# sqliteを操作するカーソルを準備
curs = conn.cursor()

# テーブルの作成とデータの挿入を行うかチェック
if os.path.exists('check.txt') == False:
    setting(curs)

# 'James'のemploy_idを取得
print(get_employ_id('James'))

### pickle
Pythonのデータをそのまま保存する

In [6]:
import pickle

class D(object):
    def __init__(self, name):
        self.name = name

data = {
    'a': [1, 2, 3],
    'b:': ('test1', 'test2'),
    'c': {'key': 'value'},
    'd': D('test')
}

# pickleファイルに保存
with open('data.pickle', 'wb') as f:
    pickle.dump(data, f)

# pickleファイルの読み込み
with open('data.pickle', 'rb') as f:
    data_load = pickle.load(f)
    print(data_load)
    print(data_load['d'].name)

{'a': [1, 2, 3], 'b:': ('test1', 'test2'), 'c': {'key': 'value'}, 'd': <__main__.D object at 0x7f4f5e591610>}
test


### MongoDB
ドキュメント型
*   Jsonで大量の情報を読み込みで使用する
*   非常に早い
*   UPDATEを行うことが頻繁になく検索のみを行う場合は優れている





```
brew tap mongodb/brew
brew install mongodb-community

pip3 install pymongo
```



In [ ]:
import datetime
from pymongo import MongoClient

# mongodbにアクセス
client = MongoClient('mongodb://localhost:27017/')

# データベースの作成
db = client['test_database']

# 登録内容をjson形式で設定
stack1 = {
    'name': 'customer1',
    'pip': ['python', 'php', 'go'],
    'info': {'os': 'mac'},
    'date': datetime.datetime.utcnow()
}

stack2 = {
    'name': 'customer2',
    'pip': ['python', 'php'],
    'info': {'os': 'windows'},
    'date': datetime.datetime.utcnow()
}


# テーブルを作成しているイメージ
db_stacks = db.stacks


# dbに保存
stacks_id = db_stacks.insert_one(stack1).inserted_id
print(stacks_id, type(stacks_id))

print('')
print(db_stacks.find_one({'_id': stacks_id}))

# idを直接指定して検索する場合は、ObjectIdを使用する
from bson.objectid import ObjectId
id = '63159ec19d5461052f1acc85'
print(db_stacks.find_one({'_id': ObjectId(id)}))

print('')

# その他の検索
name = db_stacks.find_one({'name': 'customer1'})
print(name)

print('')

pip = db_stacks.find_one({'pip': ['python', 'php', 'go']})
print(pip)

print('')

# stack2を保存
stacks_id = db_stacks.insert_one(stack2).inserted_id

# 全件検索
for stack in db_stacks.find():
    print(stack)

print('')

# 日付による全件検索
now = datetime.datetime.utcnow()
for stack in db_stacks.find({'date': {'$lt': now}}):
    print(stack)

print('')

# UPDATEを行う
db_stacks.find_one_and_update(
    {'name': 'customer1'}, {'$set': {'name': 'YYY'}}
)
name = db_stacks.find_one({'name': 'YYY'})
print(name)


# DELETEを行う
db_stacks.find_one_and_delete(
    {'name': 'YYY'}
)
name = db_stacks.find_one({'name': 'YYY'})
print(name)

### Hbase
ワイドカラム型

*   SNS系で使用されている

```
# hbaseのインストール
brew install hbase

# hbaseの再起動
sudo brew services restart hbase
or
start-hbase.sh

# shellでの起動
hbase shell

# snsテーブルにblogというカラムを作成
create 'sns', 'blog'

# snsテーブルにuser1というrowにblog:pythonにblog1という値を入れる
put 'sns', 'user1', 'blog:python', 'blog1'

# scanでテーブルを確認
scan 'sns'

# snsテーブルにuser1というrowにblog:basketballにblog2という値を入れる
put 'sns', 'user1', 'blog:basketball', 'blog2'

# 値の取得
get 'sns', 'user1'

# 列情報を取得
scan 'sns', {COLUMNS => ['blog:basketball']}

# テーブルの削除
disable 'sns'
drop 'sns'

＃ 抜け出す
quit

# hbaseの停止
sudo brew services stop hbase
or
stop-hbase.sh

# hbaseが外部アクセスを許可する
hbase thrift start

pip3 install happybase
```
snsテーブルの構造

| Row   |  blog:python  | blog:basketball |
| :---:    |    :---:        | :---:            |
| user1  |    blog1     |  blog2          |
| user2  |               |    blog4        |



In [ ]:
import happybase

connection = happybase.Connection('localhost')
connection.open()

# テーブルの作成
connection.create_table(b'sns', {'blog': dict()})

# テーブルへコネクション
table = connection.table(b'sns')

# テーブルへ値の登録
table.put(
    b'user1', {
        b'blog:python': b'user1 about python',
        b'blog:basketball': b'user1 about basketball'
    }
)

table.put(
    b'user2', {
        b'blog:basketball': b'user2 about basketball'
    }
)

# 値の取得
print(list(table.scan()))
print()
print(list(table.scan(row_prefix=b'user1')))
print()
print(list(table.scan(columns=[b'blog:basketball'])))

# テーブルの削除
connection.disable_table(b'sns')
connection.delete_table(b'sns')

### Neo4j
グラフ型

```
# Personクラスオブジェクトをjamesという変数に代入
CREATE (james:Person {name: "James"})
RETURN james

# PersonクラスオブジェクトであるjamesとLanguageクラスオブジェクトのpython、この2つを関係性を結ぶ
MATCH (james:Person {name: "James"})
CREATE (james)-[like:LIKE]->(python:Language {name: "python"})
RETURN james,like,python

# forループでjamesと値をフレンドという関係性で紐付ける
MATCH (james:Person {name: "James"})
FOREACH (name in ["Paul", "Anthony", "Wade", "Jesse"] |
    CREATE (james)-[:FRIEND]->(:Person {name:name}))

# jamesとフレンドという関係性のあるものをyourFriends変数に代入
MATCH (james:Person {name: "James"})-[:FRIEND]->(yourFriends)
RETURN james,yourFriends

# pythonとjesseを取得しjesseと友達のMichaelを作成
# MichaelはPersonとExpertオブジェクトを持ち、pythonとの関係はWORKED_WITHで結ぶ
MATCH (python:Language {name: "python"})
MATCH (jesse:Person {name: "Jesse"})
CREATE (jesse)-[:FRIEND]->(:Person:Expert {name:'Michael'})-[:WORKED_WITH]->(python)

MATCH (james {name: "James"})
MATCH (expert)-[:WORKED_WITH]->(python:Language {name: "python"})
MATCH path = shortestPath( (james)-[:FRIEND*..5]-(expert)　)
RETURN python,expert,path


pip3 install neo4j-driver
```





In [ ]:
from neo4j import GraphDatabase

driver = GraphDatabase.driver('bolt://127.0.0.1:7687', auth=('neo4j', 'password'))

def clear_db(tx):
    tx.run('MATCH (n) DETACH DELETE n')

def add_friend(tx, name, friend_name=None):
    if not friend_name:
        tx.run('CREATE (p:Person {name: $name}) RETURN p', name=name)
    return tx.run('MATCH (p:Person {name: $name})'
                  'CREATE (p)-[:FRIEND]->(:Person {name: $friend_name})',
                  name=name, friend_name=friend_name)

def print_friend(tx, name):
    for record in tx.run('MATCH (p:Person {name: $name})-[:FRIEND]->(yourFriends)'
                         'RETURN p,yourFriends', name=name):
        print(record)

with driver.session() as session:
    session.write_transaction(clear_db)
    session.write_transaction(add_friend, 'James')
    for i in ['Paul', 'Anthony', 'Wade', 'Jesse']:
        session.write_transaction(add_friend, 'James', i)
    session.read_transaction(print_friend, 'James')